# Question 1

In [1]:
def find_overlap(s1, s2):
    max_overlap = min(len(s1), len(s2))
    for i in range(max_overlap, 0, -1):
        if s1.endswith(s2[:i]):
            return s1 + s2[i:]
    return s1 + s2

def find_shortest_common_superstring(strings):
    while len(strings) > 1:
        max_overlap_len = 0
        scs_pair = (0, 1)

        for i in range(len(strings)):
            for j in range(i + 1, len(strings)):
                overlap = find_overlap(strings[i], strings[j])
                if len(overlap) > max_overlap_len:
                    max_overlap_len = len(overlap)
                    scs_pair = (i, j)

        i, j = scs_pair
        strings[i] = find_overlap(strings[i], strings[j])
        strings.pop(j)

    return strings[0]

input_strings = ["CCT", "CTT", "TGC", "TGG", "GAT", "ATT"]
shortest_common_superstring = find_shortest_common_superstring(input_strings)

# Count different shortest common superstrings
count_different_scs = input_strings.count(shortest_common_superstring)

print("Length of the shortest common superstring:", len(shortest_common_superstring))
print("Number of different shortest common superstrings:", count_different_scs)


Length of the shortest common superstring: 17
Number of different shortest common superstrings: 1


In [2]:
def generate_kmers_from_string(s, k):
    kmers = []
    for i in range(len(s) - k + 1):
        kmers.append(s[i:i + k])
    return kmers

def generate_overlap_graph(strings, k):
    overlap_graph = {}

    for s in strings:
        kmers = generate_kmers_from_string(s, k)
        suffix = kmers[0][1:]

        for kmer in kmers[1:]:
            if suffix in overlap_graph:
                overlap_graph[suffix].append(kmer)
            else:
                overlap_graph[suffix] = [kmer]

            suffix = kmer[1:]

    return overlap_graph

def generate_de_bruijn_graph(strings, k):
    de_bruijn_graph = {}

    for s in strings:
        kmers = generate_kmers_from_string(s, k)
        prefix = kmers[0][:-1]

        for kmer in kmers[1:]:
            if prefix in de_bruijn_graph:
                de_bruijn_graph[prefix].append(kmer[1:])
            else:
                de_bruijn_graph[prefix] = [kmer[1:]]

            prefix = kmer[:-1]

    return de_bruijn_graph

# Generating Overlap Graph of 3-mers
overlap_graph = generate_overlap_graph(input_strings, 3)
print("Overlap Graph:")
print(overlap_graph)

# Generating De Bruijn Graph of 3-mers
de_bruijn_graph = generate_de_bruijn_graph(input_strings, 3)
print("\nDe Bruijn Graph:")
print(de_bruijn_graph)


Overlap Graph:
{'CT': ['CTG', 'CTT', 'CTG'], 'TG': ['TGA', 'TGC', 'TGG'], 'GA': ['GAT'], 'AT': ['ATC', 'ATT'], 'TC': ['TCT'], 'TT': ['TTA', 'TTG'], 'TA': ['TAT'], 'GC': ['GCT']}

De Bruijn Graph:
{'CC': ['TG'], 'CT': ['GA', 'TA', 'GG'], 'TG': ['AT', 'CT'], 'GA': ['TC'], 'AT': ['CT', 'TG'], 'TC': ['TT'], 'TT': ['AT', 'GC'], 'TA': ['TT'], 'GC': ['TG']}


# Question 2

In [3]:
from Bio import SeqIO

def assemble_reads(file_path):
    assembled_genome = ''
    with open(file_path, 'r') as file:
        for record in SeqIO.parse(file, 'fastq'):
            assembled_genome += str(record.seq)
    return assembled_genome

def count_nucleotides(genome):
    a_count = genome.count('A')
    t_count = genome.count('T')
    c_count = genome.count('C')
    g_count = genome.count('G')

    total_bases = len(genome)

    a_percentage = (a_count / total_bases) * 100
    t_percentage = (t_count / total_bases) * 100
    c_percentage = (c_count / total_bases) * 100
    g_percentage = (g_count / total_bases) * 100

    return a_count, t_count, c_count, g_count, a_percentage, t_percentage, c_percentage, g_percentage

# Assuming the FASTQ file is in the current working directory and named "ads1_week4_reads.fq"
file_path = 'ads1_week4_reads.fq'
assembled_genome = assemble_reads(file_path)

a_count, t_count, c_count, g_count, a_percentage, t_percentage, c_percentage, g_percentage = count_nucleotides(assembled_genome)

print("Number of As in the assembled genome:", a_count)
print("Number of Ts in the assembled genome:", t_count)
print("Number of Cs in the assembled genome:", c_count)
print("Number of Gs in the assembled genome:", g_count)

print("Percentage of As in the assembled genome:", a_percentage, "%")
print("Percentage of Ts in the assembled genome:", t_percentage, "%")
print("Percentage of Cs in the assembled genome:", c_percentage, "%")
print("Percentage of Gs in the assembled genome:", g_percentage, "%")


Number of As in the assembled genome: 54873
Number of Ts in the assembled genome: 43870
Number of Cs in the assembled genome: 44924
Number of Gs in the assembled genome: 44433
Percentage of As in the assembled genome: 29.17224880382775 %
Percentage of Ts in the assembled genome: 23.322700691121746 %
Percentage of Cs in the assembled genome: 23.883040935672515 %
Percentage of Gs in the assembled genome: 23.62200956937799 %


# Question 3

In [9]:
from Bio import Entrez, SeqIO

def download_genbank_file(genbank_code):
    Entrez.email = "arnavg1808@gmail.com"  # Set your email address
    handle = Entrez.efetch(db="nucleotide", id=genbank_code, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record.seq

# Replace 'U01317.1' with the actual GenBank code you want to download
genbank_code = 'U01317.1'
sequence = download_genbank_file(genbank_code)
#print(sequence)

In [10]:
def count_occurrences(sequence, pattern):
    return sequence.count(pattern)

def reverse_complement(sequence):
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement_dict[base] for base in reversed(sequence))

pattern = "CTTAGAACGGAAATCTTAGT"
reverse_complement_pattern = reverse_complement(pattern)

occurrences = count_occurrences(sequence, pattern) + count_occurrences(sequence, reverse_complement_pattern)
print("Total occurrences of the pattern and its reverse complement:", occurrences)


Total occurrences of the pattern and its reverse complement: 0


In [13]:
def naive_exact_matching(text, pattern):
    occurrences = []
    comparisons = 0
    n = len(text)
    m = len(pattern)

    for i in range(n - m + 1):
        match = True
        for j in range(m):
            if text[i + j] != pattern[j]:
                comparisons += 1
                match = False
                break
        if match:
            occurrences.append(i)

    return occurrences, comparisons

# Time the Naive exact matching algorithm and count character comparisons performed
import time

start_time = time.time()
occurrences_naive, comparisons_naive = naive_exact_matching(sequence, pattern)
occurrences_reverse_complement, comparisons_reverse_complement = naive_exact_matching(sequence, reverse_complement_pattern)
end_time = time.time()
time_taken_naive = end_time - start_time

print("Occurrences using Naive exact matching:", len(occurrences_naive) + len(occurrences_reverse_complement))
print("Time taken by Naive exact matching algorithm:", time_taken_naive, "seconds")
print("Character comparisons performed by Naive exact matching algorithm:", comparisons_naive + comparisons_reverse_complement)


Occurrences using Naive exact matching: 0
Time taken by Naive exact matching algorithm: 0.19381952285766602 seconds
Character comparisons performed by Naive exact matching algorithm: 146578


# Question 4

In [19]:
from Bio import Entrez, SeqIO

def download_gene_sequence(gene_id):
    Entrez.email = "arnavg1808@gmail.comm"  # Set your email address
    handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record.seq

# Replace "123456" and "789012" with the actual gene IDs for rats and mice, respectively
rat_gene_id = "65262"
mouse_gene_id = "11946"

# Download the gene sequences for rats and mice
rat_gene_sequence = download_gene_sequence(rat_gene_id)
#mouse_gene_sequence = download_gene_sequence(mouse_gene_id)

print("Rat ATP Synthase Subunit gene sequence:", rat_gene_sequence)
#print("Mouse ATP Synthase Subunit gene sequence:", mouse_gene_sequence)


Rat ATP Synthase Subunit gene sequence: TCCAAGATCGAGCGGCGGAAGATTATGGAACAATCCCCGGACATGCACAACGCCGAGATCTCCAAGCGCCTGGGGAAACGCTGGAAAATGCTCAACGACAGCGAGAAAATCCCCTTCATCAGGGAAGCCGAGCGGCTGCGGCTCAAACACATGGCCGACTAC


In [20]:
def calculate_edit_distance(seq1, seq2, transversion_score, transition_score, gap_score):
    if len(seq1) > len(seq2):
        seq1, seq2 = seq2, seq1

    edit_distance = 0
    for base1, base2 in zip(seq1, seq2):
        if base1 != base2:
            if (base1, base2) in [('A', 'G'), ('G', 'A'), ('C', 'T'), ('T', 'C')]:  # Transversions
                edit_distance += transversion_score
            else:  # Transitions
                edit_distance += transition_score

    # For remaining characters in the longer sequence
    edit_distance += gap_score * (len(seq2) - len(seq1))

    return edit_distance

# Gene sequences for rats and mice (replace with the actual sequences)
gene_rat = "TCCAAGATCGAGCGGCGGAAGATTATGGAACAATCCCCGGACATGCACAACGCCGAGATCTCCAAGCGCCTGGGGAAACGCTGGAAAATGCTCAACGACAGCGAGAAAATCC\
CCTTCATCAGGGAAGCCGAGCGGCTGCGGCTCAAACACATGGCCGACTAC"
gene_mouse = "TCCAGTCCCTCGGCGGAAGATTATGGAACAATCCCCGGACATGCACAACGCCGAGATCAGGCCTAAGCGGCTAAAGCGCTGGAAAATGCTCAACGACAGCGAGAAAATCC\
CCTTCATCAGGGAAGCCGAGCGGCTGCGGCTCAAACACATGGCCGACTAC"

# Method 1: Transversions are given score of 2, transitions of 2, gaps are given score of 2
edit_distance_method1 = calculate_edit_distance(gene_rat, gene_mouse, transversion_score=2, transition_score=2, gap_score=2)

# Method 2: Transversions are given score of 2, transitions of 4, gaps are given score of 4
edit_distance_method2 = calculate_edit_distance(gene_rat, gene_mouse, transversion_score=2, transition_score=4, gap_score=4)

# Method 3: Transversions are given score of 4, transitions of 2, gaps are given score of 8
edit_distance_method3 = calculate_edit_distance(gene_rat, gene_mouse, transversion_score=4, transition_score=2, gap_score=8)

# Method 4: Find the hamming distance between the two (splice the larger string to take the first n elements)
shorter_length = min(len(gene_rat), len(gene_mouse))
hamming_distance = sum(base1 != base2 for base1, base2 in zip(gene_rat[:shorter_length], gene_mouse[:shorter_length]))

print("Edit distance (Method 1):", edit_distance_method1)
print("Edit distance (Method 2):", edit_distance_method2)
print("Edit distance (Method 3):", edit_distance_method3)
print("Hamming distance (Method 4):", hamming_distance)


Edit distance (Method 1): 240
Edit distance (Method 2): 398
Edit distance (Method 3): 334
Hamming distance (Method 4): 118


# Question 5

In [21]:
from Bio import Entrez, SeqIO

def download_cds_file(accession_number):
    Entrez.email = "arnavg1808@gmail.com"  # Set your email address
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

# Replace 'NC_045512.2' with the actual accession number of the coronavirus genome
accession_number = 'NC_045512.2'
coronavirus_record = download_cds_file(accession_number)


In [23]:
def print_cds_features(record):
    for feature in record.features:
        if feature.type == "CDS":
            gene_name = feature.qualifiers.get("gene", ["Unknown Gene"])[0]
            print("Gene Name:", gene_name)

# Print all CDS features and their gene names
print_cds_features(coronavirus_record)


Gene Name: ORF1ab
Gene Name: ORF1ab
Gene Name: S
Gene Name: ORF3a
Gene Name: E
Gene Name: M
Gene Name: ORF6
Gene Name: ORF7a
Gene Name: ORF7b
Gene Name: ORF8
Gene Name: N
Gene Name: ORF10


In [24]:
def extract_gene_sequence(record, gene_name):
    for feature in record.features:
        if feature.type == "CDS" and "gene" in feature.qualifiers and feature.qualifiers["gene"][0] == gene_name:
            return feature.location.extract(record.seq)

# Replace "ORF10" with the correct gene name for the helicase enzyme
helicase_gene_name = "ORF10"

# Extract the DNA sequence of the helicase enzyme and its reverse complement
helicase_dna_sequence = extract_gene_sequence(coronavirus_record, helicase_gene_name)
reverse_complement_helicase_dna_sequence = helicase_dna_sequence.reverse_complement()

print("Helicase DNA sequence:", helicase_dna_sequence)
print("Reverse complement of Helicase DNA sequence:", reverse_complement_helicase_dna_sequence)


Helicase DNA sequence: ATGGGCTATATAAACGTTTTCGCTTTTCCGTTTACGATATATAGTCTACTCTTGTGCAGAATGAATTCTCGTAACTACATAGCACAAGTAGATGTAGTTAACTTTAATCTCACATAG
Reverse complement of Helicase DNA sequence: CTATGTGAGATTAAAGTTAACTACATCTACTTGTGCTATGTAGTTACGAGAATTCATTCTGCACAAGAGTAGACTATATATCGTAAACGGAAAAGCGAAAACGTTTATATAGCCCAT


In [25]:
def divide_into_k_mers(sequence, k):
    return [sequence[i:i + k] for i in range(len(sequence) - k + 1)]

orf7a_sequence = extract_protein_sequence(coronavirus_record, "ORF7a")
k_mers_8 = divide_into_k_mers(orf7a_sequence, 8)
k_mers_9 = divide_into_k_mers(orf7a_sequence, 9)
k_mers_10 = divide_into_k_mers(orf7a_sequence, 10)
k_mers_11 = divide_into_k_mers(orf7a_sequence, 11)
k_mers_12 = divide_into_k_mers(orf7a_sequence, 12)

assembled_8_mers = "".join(k_mers_8)
assembled_9_mers = "".join(k_mers_9)
assembled_10_mers = "".join(k_mers_10)
assembled_11_mers = "".join(k_mers_11)
assembled_12_mers = "".join(k_mers_12)

print("Are the assembled sequences the same?")
print(assembled_8_mers == assembled_9_mers == assembled_10_mers == assembled_11_mers == assembled_12_mers)


Are the assembled sequences the same?
False


In [28]:
def extract_gene_sequence(record, gene_name):
    for feature in record.features:
        if feature.type == "CDS" and "gene" in feature.qualifiers and feature.qualifiers["gene"][0] == gene_name:
            return feature.location.extract(record.seq)

# Replace "S" with the correct gene name for the surface glycoprotein (spike) gene
surface_glycoprotein_gene_name = "S"

# Extract the DNA sequence of the surface glycoprotein (spike) gene
surface_glycoprotein_sequence = extract_gene_sequence(coronavirus_record, surface_glycoprotein_gene_name)

# Generate 20 random 15-mers from the surface glycoprotein gene
import random

random.seed(42)
random_15_mers = [surface_glycoprotein_sequence[i:i + 15] for i in random.sample(range(len(surface_glycoprotein_sequence) - 15 + 1), 20)]

# Perform naive exact matching with two mismatches against the RNA-dependent RNA polymerase gene
rna_polymerase_gene_name = "ORF1ab"
rna_polymerase_sequence = extract_gene_sequence(coronavirus_record, rna_polymerase_gene_name)

def naive_exact_matching_with_mismatches(text, pattern, max_mismatches):
    matches = []
    n = len(text)
    m = len(pattern)

    for i in range(n - m + 1):
        mismatches = sum(text[i + j] != pattern[j] for j in range(m))
        if mismatches <= max_mismatches:
            matches.append((i, mismatches))

    return matches

# Perform naive exact matching with two mismatches for each random 15-mer
random_matches = [naive_exact_matching_with_mismatches(rna_polymerase_sequence, random_15_mer, 2) for random_15_mer in random_15_mers]

print("Matches of the 20 random 15-mers in RNA-dependent RNA polymerase gene:")
print(random_matches)


Matches of the 20 random 15-mers in RNA-dependent RNA polymerase gene:
[[], [(19980, 2)], [], [], [], [], [], [], [(16144, 1)], [], [], [], [], [], [], [(15669, 1)], [], [], [], []]


# Question 6


https://drive.google.com/file/d/11cL9JyljKuSdk2IXQiGmTF6QX-7YzNzc/view?usp=sharing